# ++ Carregar dados históricos da Bovespa em Dataframe para análise ++

In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Wed Nov 16 19:28:38 2022'

In [2]:
import sys
from pandas import Series
import pandas as pd
from datetime import date
pd.__version__
from zipfile import ZipFile

In [3]:
import matplotlib as mat
mat.__version__

'3.4.3'

In [4]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [5]:
from IPython.display import FileLink, FileLinks
FileLink('Manipulação CSV BB.ipynb')

C:\Users\F0435280\OneDrive - Banco do Brasil S.A\Área de Trabalho\Alexandre\StockManager-main\Manipulação CSV BB.ipynb

In [6]:
# Setando opção do Pandas para mostrar todas as linhas
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [7]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
def leituraArquivos(periodo):
    arq = 'COTAHIST_' + periodo

    DTEXCH, CODNEG, PREABE, PREMAX, PREMIN, PREULT, VOLTOT = ([] for i in range(7)) 
    
    with ZipFile('arquivos/' + arq + '.ZIP') as myzip:
        with myzip.open(arq + '.TXT') as myfile:
            for line in myfile:
                if (int(line.decode('utf-8')[0:2]) == 1) and (line.decode('utf-8')[10:12] == '02'):
                    DTEXCH.append(line.decode('utf-8')[2:10])
                    CODNEG.append(line.decode('utf-8')[12:24].rstrip())
                    PREABE.append(int(line.decode('utf-8')[56:69]) / 100)
                    PREMAX.append(int(line.decode('utf-8')[69:82]) / 100)
                    PREMIN.append(int(line.decode('utf-8')[82:95]) / 100)
                    PREULT.append(int(line.decode('utf-8')[108:121]) / 100)
                    VOLTOT.append(int(line.decode('utf-8')[170:188]) / 100)

    df_origem = pd.DataFrame(
        {"cdAcao":CODNEG
        ,"dtPregao":pd.to_datetime(DTEXCH, format="%Y%m%d", errors="ignore")
        ,"vrFech":PREULT
        ,"vrVolume":VOLTOT
        ,"vrMaxDia":PREMAX
        ,"vrMinDia":PREMIN
        ,"vrAbert":PREABE
        }
    ).sort_values(["cdAcao", "dtPregao"], ascending=True)

    return(df_origem)

In [9]:
def carrega_dados():
    df = leituraArquivos('A2022')

#    df = pd.concat([df_112021, df_122021, df])

    df["pcVar"], df["pcMaxDia"], df["pcMinDia"], df["pcAbert"] = [
         ((df.vrFech   / df.vrFech.shift(1)) -1) *100
        ,((df.vrMaxDia / df.vrFech.shift(1)) -1) *100
        ,((df.vrMinDia / df.vrFech.shift(1)) -1) *100
        ,((df.vrAbert  / df.vrFech.shift(1)) -1) *100
        ]
    
    df["ic05"], df["ic10"], df["ic15"], df["ic20"], df["ic25"], df["ic30"] = [
         df.apply(condicao05, axis=1)
        ,df.apply(condicao10, axis=1)
        ,df.apply(condicao15, axis=1)
        ,df.apply(condicao20, axis=1)
        ,df.apply(condicao25, axis=1)
        ,df.apply(condicao30, axis=1)
        ]

    return df

In [10]:
def condicao05(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 0.5) else 0
def condicao10(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1) else 0
def condicao15(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1.5) else 0
def condicao20(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2) else 0
def condicao25(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2.5) else 0
def condicao30(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 3) else 0

In [11]:
# Exclui header e trailer
# df = df.drop(df.index[[0, -1]])

In [13]:
def buscaPeriodos(qt_dias):
    return df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[qt_dias])].sort_values(["cdAcao", "dtPregao"], ascending=False)

In [14]:
def somatorioPcMaxDia(df_ent, pc, index_name):
    return df_ent.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>pc).sum()).reset_index(name=index_name)

In [15]:
def buscaMedia(df_ent, coluna, index_name):
    return df_ent.groupby("cdAcao")[coluna].agg("mean").reset_index(name=index_name)

In [12]:
df = carrega_dados()

In [16]:
# Seleciona últimos x dias de cada ação
df25d = buscaPeriodos(24)
df20d = buscaPeriodos(19)
df15d = buscaPeriodos(14)
df10d = buscaPeriodos(9)
df5d  = buscaPeriodos(4)

In [18]:
df05 = somatorioPcMaxDia(df25d, 0.5, "0.5%")
df10 = somatorioPcMaxDia(df25d, 1.0, "resultado")
df15 = somatorioPcMaxDia(df25d, 1.5, "resultado")
df20 = somatorioPcMaxDia(df25d, 2.0, "resultado")
df25 = somatorioPcMaxDia(df25d, 2.5, "resultado")
df30 = somatorioPcMaxDia(df25d, 3.0, "resultado")
dfVol     = buscaMedia(df25d, "vrVolume", "vol")
dfVrFech  = buscaMedia(df25d, "vrFech", "vrFech")
dfPcAbert = buscaMedia(df25d, "pcAbert", "pcAbert")

In [19]:
df05_5d = somatorioPcMaxDia(df5d, 0.5, "0.5%")
df10_5d = somatorioPcMaxDia(df5d, 1.0, "resultado")
df15_5d = somatorioPcMaxDia(df5d, 1.5, "resultado")
df20_5d = somatorioPcMaxDia(df5d, 2.0, "resultado")
df25_5d = somatorioPcMaxDia(df5d, 2.5, "resultado")
df30_5d = somatorioPcMaxDia(df5d, 3.0, "resultado")
dfVol_5d     = buscaMedia(df5d, "vrVolume", "vol")
dfVrFech_5d  = buscaMedia(df5d, "vrFech", "vrFech")
dfPcAbert_5d = buscaMedia(df5d, "pcAbert", "pcAbert")

In [20]:
df05_10d = somatorioPcMaxDia(df10d, 0.5, "0.5%")
df10_10d = somatorioPcMaxDia(df10d, 1.0, "resultado")
df15_10d = somatorioPcMaxDia(df10d, 1.5, "resultado")
df20_10d = somatorioPcMaxDia(df10d, 2.0, "resultado")
df25_10d = somatorioPcMaxDia(df10d, 2.5, "resultado")
df30_10d = somatorioPcMaxDia(df10d, 3.0, "resultado")
dfVol_10d     = buscaMedia(df10d, "vrVolume", "vol")
dfVrFech_10d  = buscaMedia(df10d, "vrFech", "vrFech")
dfPcAbert_10d = buscaMedia(df10d, "pcAbert", "pcAbert")

In [21]:
df05_15d = somatorioPcMaxDia(df15d, 0.5, "0.5%")
df10_15d = somatorioPcMaxDia(df15d, 1.0, "resultado")
df15_15d = somatorioPcMaxDia(df15d, 1.5, "resultado")
df20_15d = somatorioPcMaxDia(df15d, 2.0, "resultado")
df25_15d = somatorioPcMaxDia(df15d, 2.5, "resultado")
df30_15d = somatorioPcMaxDia(df15d, 3.0, "resultado")
dfVol_15d     = buscaMedia(df15d, "vrVolume", "vol")
dfVrFech_15d  = buscaMedia(df15d, "vrFech", "vrFech")
dfPcAbert_15d = buscaMedia(df15d, "pcAbert", "pcAbert")

In [22]:
df05_20d = somatorioPcMaxDia(df20d, 0.5, "0.5%")
df10_20d = somatorioPcMaxDia(df20d, 1.0, "resultado")
df15_20d = somatorioPcMaxDia(df20d, 1.5, "resultado")
df20_20d = somatorioPcMaxDia(df20d, 2.0, "resultado")
df25_20d = somatorioPcMaxDia(df20d, 2.5, "resultado")
df30_20d = somatorioPcMaxDia(df20d, 3.0, "resultado")
dfVol_20d     = buscaMedia(df20d, "vrVolume", "vol")
dfVrFech_20d  = buscaMedia(df20d, "vrFech", "vrFech")
dfPcAbert_20d = buscaMedia(df20d, "pcAbert", "pcAbert")

In [23]:
df05["1.0%"], df05["1.5%"], df05["2.0%"], df05["2.5%"], df05["3.0%"] , df05["AvgVol"], df05["AvgVrFech"], df05["AvgPcAbert"] = [
    df10["resultado"],df15["resultado"],df20["resultado"],df25["resultado"],df30["resultado"],dfVol["vol"],dfVrFech["vrFech"],dfPcAbert["pcAbert"]]
df05_5d["1.0%"], df05_5d["1.5%"], df05_5d["2.0%"], df05_5d["2.5%"], df05_5d["3.0%"] , df05_5d["AvgVol"], df05_5d["AvgVrFech"], df05_5d["AvgPcAbert"] = [
    df10_5d["resultado"],df15_5d["resultado"],df20_5d["resultado"],df25_5d["resultado"],df30_5d["resultado"],dfVol_5d["vol"],dfVrFech_5d["vrFech"],dfPcAbert_5d["pcAbert"]]
df05_10d["1.0%"], df05_10d["1.5%"], df05_10d["2.0%"], df05_10d["2.5%"], df05_10d["3.0%"] , df05_10d["AvgVol"], df05_10d["AvgVrFech"], df05_10d["AvgPcAbert"] = [
    df10_10d["resultado"],df15_10d["resultado"],df20_10d["resultado"],df25_10d["resultado"],df30_10d["resultado"],dfVol_10d["vol"],dfVrFech_10d["vrFech"],dfPcAbert_10d["pcAbert"]]
df05_15d["1.0%"], df05_15d["1.5%"], df05_15d["2.0%"], df05_15d["2.5%"], df05_15d["3.0%"] , df05_15d["AvgVol"], df05_15d["AvgVrFech"], df05_15d["AvgPcAbert"] = [
    df10_15d["resultado"],df15_15d["resultado"],df20_15d["resultado"],df25_15d["resultado"],df30_15d["resultado"],dfVol_15d["vol"],dfVrFech_15d["vrFech"],dfPcAbert_15d["pcAbert"]]
df05_20d["1.0%"], df05_20d["1.5%"], df05_20d["2.0%"], df05_20d["2.5%"], df05_20d["3.0%"] , df05_20d["AvgVol"], df05_20d["AvgVrFech"], df05_20d["AvgPcAbert"] = [
    df10_20d["resultado"],df15_20d["resultado"],df20_20d["resultado"],df25_20d["resultado"],df30_20d["resultado"],dfVol_20d["vol"],dfVrFech_20d["vrFech"],dfPcAbert_20d["pcAbert"]]

In [24]:
df05     =     df05.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_5d  =  df05_5d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_10d = df05_10d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_15d = df05_15d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_20d = df05_20d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)

In [25]:
pd.set_option('display.max_rows', None)
# df06 = df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 20) & (df05["AvgVrFech"] > 5)]

In [26]:
df25d['vrVolume'] = df25d['vrVolume'].astype(int)
df25d['vrVolume'] = df25d['vrVolume'].map('{:,}'.format)

### 25 dias, order by 1%

In [27]:
df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 21) & (df05["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
489,KLBN11,24,22,15,13,10,8,"146,477,531.72",21.18,0.01
408,GFSA3,23,21,19,16,15,13,"12,974,184.64",7.65,0.18
356,ENAT3,21,21,19,10,8,6,"23,023,487.12",15.34,-0.52
113,BEEF3,22,21,18,13,12,8,"101,180,165.12",13.89,-0.13
301,CURY3,21,21,18,11,10,7,"22,525,911.72",12.10,-0.11
642,PTBL3,21,21,17,13,11,9,"16,505,255.52",12.08,-0.34
757,TGMA3,22,21,15,12,8,6,"7,789,171.56",21.64,-0.11


### 20 dias, order by 1%

In [28]:
df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["1.0%"] >= 17) & (df05_20d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
168,ENAT3,18,18,16,9,7,5,"23,553,854.40",15.28,-0.57
200,GFSA3,19,17,16,13,12,10,"13,696,952.75",7.56,0.19
325,QUAL3,18,17,15,14,9,9,"38,165,964.25",7.44,-0.17
423,YDUQ3,18,17,15,12,11,9,"123,829,593.80",13.87,0.25
339,ROMI3,18,17,15,12,11,7,"12,970,605.55",16.12,-0.01
335,RDOR3,19,17,15,11,9,8,"231,133,904.20",30.91,-0.14
399,UGPA3,19,17,14,14,9,9,"125,679,000.30",13.34,-0.23
167,EMBR3,17,17,14,11,9,7,"101,750,416.95",13.08,-0.18
140,CURY3,17,17,14,10,9,7,"22,458,302.00",12.07,-0.12
41,BEEF3,18,17,14,10,9,6,"97,972,861.45",14.04,-0.17


### 15 dias, order by 1%

In [29]:
df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["1.5%"] >= 12) & (df05_15d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
331,ROMI3,14,14,12,10,10,7,"14,735,663.93",16.26,-0.11
415,YDUQ3,14,13,13,10,9,7,"134,294,877.53",14.24,0.26
317,QUAL3,13,13,12,11,8,8,"37,003,377.00",7.43,-0.33
196,GFSA3,14,13,12,10,10,9,"14,755,471.73",7.64,-0.11
46,BLAU3,14,13,12,9,8,6,"12,122,125.67",32.29,0.25
18,AMER3,12,12,12,10,10,10,"324,831,355.00",14.60,-0.91
239,LIGT3,12,12,12,10,8,8,"28,002,317.33",5.62,-0.29
285,ONCO3,12,12,12,9,5,5,"7,582,713.67",7.84,-0.20


### 10 dias, order by 1%

In [30]:
df05_10d.loc[(df05_10d["AvgVol"] > 6000000) & (df05_10d["1.0%"] >= 9) & (df05_10d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
384,UGPA3,10,10,8,8,4,4,"137,469,237.80",13.75,-0.52
311,QUAL3,9,9,9,9,7,7,"41,964,100.20",7.53,-0.38
136,CVCB3,9,9,9,8,6,5,"118,043,824.40",6.67,-0.21
193,GFSA3,9,9,9,7,7,6,"15,087,683.40",7.80,-0.09
317,RAPT4,9,9,9,7,5,3,"23,645,594.30",9.91,-0.32
356,SULA11,10,9,8,7,6,5,"97,302,760.60",24.15,-0.33
325,ROMI3,9,9,8,6,6,3,"12,020,315.00",16.40,-0.04
45,BLAU3,9,9,8,6,5,4,"12,935,359.20",32.64,0.55
54,BOAS3,9,9,8,6,5,4,"13,146,541.50",5.85,-0.40
321,RDOR3,10,9,8,6,5,4,"266,974,980.70",32.46,-0.64


### 5 dias, order by 1%

In [31]:
df05_5d.loc[(df05_5d["AvgVol"] > 6000000) & (df05_5d["1.0%"] >= 5) & (df05_5d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
39,BEEF3,5,5,5,4,4,3,"124,243,587.20",14.55,-0.75
183,GGBR4,5,5,5,4,3,3,"632,376,521.80",29.32,0.25
187,GOAU4,5,5,4,4,3,3,"169,572,125.20",12.55,0.15
370,UGPA3,5,5,4,4,2,2,"174,539,025.20",13.60,-0.45
343,SULA11,5,5,4,3,3,2,"116,171,843.80",23.91,-0.28
149,EMBR3,5,5,4,3,2,2,"151,246,628.60",13.43,-0.41
348,TAEE11,5,5,4,2,2,1,"124,858,016.20",39.33,0.41
346,SUZB3,5,5,3,3,3,2,"421,783,779.80",56.59,0.17
217,KLBN11,5,5,3,3,2,2,"154,698,723.20",23.00,-0.28


### Consulta ação específica

In [32]:
df25d.loc[(df25d["cdAcao"] == "GFSA3")].replace(0, "")

,cdAcao,dtPregao,vrFech,vrVolume,vrMaxDia,vrMinDia,vrAbert,pcVar,pcMaxDia,pcMinDia,pcAbert,ic05,ic10,ic15,ic20,ic25,ic30
171431,GFSA3,2022-11-14,6.94,"16,938,907",6.94,6.29,6.55,6.61,6.61,-3.38,0.61,1,1,1,1,1,1
171059,GFSA3,2022-11-11,6.51,"27,170,639",7.39,6.49,7.16,-10.45,1.65,-10.73,-1.51,1,1,1,,,
170695,GFSA3,2022-11-10,7.27,"12,525,224",8.09,7.26,8.09,-10.58,-0.49,-10.70,-0.49,,,,,,
170331,GFSA3,2022-11-09,8.13,"8,860,748",8.25,7.90,8.11,0.25,1.73,-2.59,,1,1,1,,,
169969,GFSA3,2022-11-08,8.11,"15,154,056",8.55,7.79,8.09,0.37,5.82,-3.59,0.12,1,1,1,1,1,1
169607,GFSA3,2022-11-07,8.08,"10,679,853",8.65,8.00,8.24,-1.94,4.98,-2.91,,1,1,1,1,1,1
169240,GFSA3,2022-11-04,8.24,"8,841,445",8.71,8.24,8.59,-2.94,2.59,-2.94,1.18,1,1,1,1,1,
168872,GFSA3,2022-11-03,8.49,"12,858,078",8.79,8.17,8.24,1.56,5.14,-2.27,-1.44,1,1,1,1,1,1
168503,GFSA3,2022-11-01,8.36,"18,269,445",8.42,7.73,7.93,6.36,7.12,-1.65,0.89,1,1,1,1,1,1
168128,GFSA3,2022-10-31,7.86,"19,578,439",8.07,7.42,7.64,2.61,5.35,-3.13,-0.26,1,1,1,1,1,1


In [33]:
# dfDtMin = df5d.loc[(df["dtPregao"] == dt_min_5d) & (df["cdAcao"] == 'PTBL3')]
dt_min_5d  = df5d["dtPregao"].min()
dt_min_25d = df25d["dtPregao"].min()
dt_max     = df5d["dtPregao"].max()
dfDtMin5d  = df5d.loc[(df5d["dtPregao"] == dt_min_5d)].set_index(["cdAcao"])
dfDtMin25d = df25d.loc[(df25d["dtPregao"] == dt_min_25d)].set_index(["cdAcao"])
dfDtMax    = df5d.loc[(df5d["dtPregao"] == dt_max)].set_index(["cdAcao"])

In [34]:
 dfPc5d = pd.DataFrame({
         "dtInicio":dfDtMin5d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin5d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin5d["vrFech"]) / dfDtMin5d["vrFech"]) * 100
    })

In [35]:
dfPc5d.sort_values(["pcPeriodo"], ascending=False)

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo
cdAcao,,,,,
GGBR3,2022-11-08,2022-11-14,22.37,26.30,17.57
GGBR4,2022-11-08,2022-11-14,27.24,31.20,14.54
OSXB3,2022-11-08,2022-11-14,5.86,6.60,12.63
VALE3,2022-11-08,2022-11-14,74.05,83.31,12.51
GOAU4,2022-11-08,2022-11-14,11.90,13.27,11.51
CSNA3,2022-11-08,2022-11-14,13.57,14.95,10.17
BAZA3,2022-11-08,2022-11-14,49.05,54.00,10.09
GEPA3,2022-11-08,2022-11-14,22.00,24.00,9.09
WLMM4,2022-11-08,2022-11-14,24.90,27.00,8.43


In [36]:
 dfPc25d = pd.DataFrame({
         "dtInicio":dfDtMin25d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin25d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin25d["vrFech"]) / dfDtMin25d["vrFech"]) * 100
    })

In [37]:
dfPc25d.sort_values(["pcPeriodo"], ascending=False)

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo
cdAcao,,,,,
MNDL3,2022-10-07,2022-11-14,30.20,40.00,32.45
VIVR3,2022-10-07,2022-11-14,0.50,0.65,30.00
GGBR3,2022-10-07,2022-11-14,20.90,26.30,25.84
KLBN4,2022-10-07,2022-11-14,3.75,4.70,25.33
SUZB3,2022-10-07,2022-11-14,46.39,57.74,24.47
KLBN11,2022-10-07,2022-11-14,18.82,23.33,23.96
GGBR4,2022-10-07,2022-11-14,25.58,31.20,21.97
WEGE3,2022-10-07,2022-11-14,33.35,40.66,21.92
GOAU4,2022-10-07,2022-11-14,11.03,13.27,20.31


In [38]:
### 25 dias, Order by 0.5% # df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5% # df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5% # df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

## Definindo valores para compra

In [39]:
# Valor disponível para cmp / 1000
# Valor da ação no leilão
vr_corretagem = 9.8
emolumentos = 0.000325
acao = 'LIGT3'
vr_disp = 110
vr_acao_leilao = 1148 / 100
vr_qtd = round((vr_disp * 1000) / (vr_acao_leilao * 100)) * 100

In [40]:
vr_cmp = round(vr_acao_leilao * 1.014, 2)

vr_tot_cmp = vr_acao_leilao * vr_qtd
corret_cmp = vr_corretagem + (vr_tot_cmp * emolumentos)
vr_tot_liq_cmp = vr_tot_cmp + corret_cmp

vr_acao_vnd_10 = float('{0:.2f}'.format(vr_acao_leilao * 1.0108 + 0.01))
vr_tot_vnd_10 = vr_acao_vnd_10 * vr_qtd
corret_vnd_10 = vr_corretagem + (vr_tot_vnd_10 * emolumentos)
vr_tot_liq_vnd_10 = vr_tot_vnd_10 - corret_vnd_10

vr_acao_vnd_15 = float('{0:.2f}'.format(vr_acao_leilao * 1.0158 + 0.01))
vr_tot_vnd_15 = vr_acao_vnd_15 * vr_qtd
corret_vnd_15 = vr_corretagem + (vr_tot_vnd_15 * emolumentos)
vr_tot_liq_vnd_15 = vr_tot_vnd_15 - corret_vnd_15

In [41]:
d = {'Compra':     ['Quantidade', 'Valor Leilão', 'Total', 'Total líquido', 'Corretagem Compra'],
     ' ':          [vr_qtd, vr_acao_leilao, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
     'Venda 1,0%': ['Valor compra', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,0%':       [vr_cmp, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
     'Venda 1,5%': [' ', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,5%':       [' ', vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
    }
df_calc = pd.DataFrame(data=d)
df_calc

,Compra,,"Venda 1,0%","1,0%","Venda 1,5%","1,5%"
0,Quantidade,"9,600.00",Valor compra,11.64,,
1,Valor Leilão,11.48,Valor,11.61,Valor,11.67
2,Total,"110,208.00",Total,"111,456.00",Total,"112,032.00"
3,Total líquido,"110,253.62",Total líquido,"111,409.98",Total líquido,"111,985.79"
4,Corretagem Compra,45.62,Corretagem Venda,46.02,Corretagem Venda,46.21


In [42]:
d2 = {'Index':      ['Quantidade', 'Valor', 'Total', 'Total líquido', 'Corretagem'],
      'Compra':     [vr_qtd, vr_cmp, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
      'Venda 1.0%': [vr_qtd, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
      'Venda 1.5%': [vr_qtd, vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
     }
df_calc2 = pd.DataFrame(data=d2)
df_calc2.set_index('Index')
df_calc2

,Index,Compra,Venda 1.0%,Venda 1.5%
0,Quantidade,"9,600.00","9,600.00","9,600.00"
1,Valor,11.64,11.61,11.67
2,Total,"110,208.00","111,456.00","112,032.00"
3,Total líquido,"110,253.62","111,409.98","111,985.79"
4,Corretagem,45.62,46.02,46.21


In [43]:
time.asctime( time.localtime(time.time()) )

'Wed Nov 16 19:29:03 2022'